In [3]:
import pandas as pd

In [6]:
df = pd.read_csv('data.csv')
df.head()

,Converted,Points,Exp_Cost,23_Cost,Max_Cost
0,QB01,461.60,50,51,51
1,QB02,436.63,47,50,50
2,QB03,413.86,44,49,49
3,QB04,391.19,41,48,48
4,QB05,366.90,34,37,37
